In [1]:
import json
import sys
import numpy as np
from pathlib import Path

from s4hci.utils.data_handling import load_adi_data, save_as_fits
from s4hci.models.psf_subtraction import S4
from s4hci.models.noise import S4Noise
from s4hci.utils.logging import print_message, setup_logger

from applefy.utils.fake_planets import add_fake_planets

# All files we need

In [2]:
# 1.) Load the arguments
dataset_hdf5_file = "/fast/mbonse/s4/30_data/HD22049_304_199_C-0065_C_.hdf5"
reg_lambda = 1934
s4_work_dir = "/fast/mbonse/s4/70_results/03_contrast_grid/fine_tune_very_long/"
saved_model = "/fast/mbonse/s4/70_results/03_contrast_grid/finished/HD22049_304_199_C-0065_C_/results/0000/models/noise_model_closed_form.pkl"

# Load the data

In [3]:
science_data, raw_angles, raw_psf_template_data = \
    load_adi_data(
        hdf5_dataset=dataset_hdf5_file,
        data_tag="object",
        psf_template_tag="psf_template",
        para_tag="header_object/PARANG")

science_data = science_data[:, 12:-12, 12:-12]

# Background subtraction of the PSF template
psf_template_data = np.median(raw_psf_template_data, axis=0)
psf_template_data = psf_template_data - np.min(psf_template_data)

In [4]:
data_with_fake_planet = science_data

# Create and restore the model

In [5]:
s4_model = S4(
    data_cube=data_with_fake_planet,
    parang=raw_angles,
    psf_template=psf_template_data,
    noise_noise_cut_radius_psf=4.0,
    noise_mask_radius=5.5,
    device=0,
    noise_lambda_init=reg_lambda,
    noise_normalization="robust",
    planet_convolve_second=True,
    planet_use_up_sample=1,
    work_dir=s4_work_dir,
    verbose=True)

Creating right reason mask ... [DONE]


In [6]:
s4_noise = S4Noise.load(saved_model)

Creating right reason mask ... [DONE]


In [7]:
s4_model.noise_model = s4_noise.float()

# Compute one test residual

In [8]:
raw_residual_file = s4_model.residuals_dir / \
        Path("01_Residual_Raw_mean.fits")

In [9]:
residual_before_fine_tuning = s4_model.compute_residual(
    account_for_planet=False,
    combine="mean")

save_as_fits(
    residual_before_fine_tuning,
    raw_residual_file,
    overwrite=True)

# Fine-tune the model

In [10]:
s4_model.fine_tune_model(
    num_epochs=500,
    learning_rate_planet=1e-3,
    learning_rate_noise=1e-6,
    fine_tune_noise_model=True,
    lean_planet_model=False,
    rotation_grid_down_sample=10,
    upload_rotation_grid=True,
    batch_size=10000)

# Create a residual
tmp_residual_file = s4_model.residuals_dir / \
    Path("01_Residual_Raw_mean_500.fits")

tmp_residual = s4_model.compute_residual(
    account_for_planet=False,
    combine="mean")

save_as_fits(
    tmp_residual,
    tmp_residual_file,
    overwrite=True)

  0%|          | 0/500 [00:00<?, ?it/s]

# Run the same but with BS -1

In [11]:
s4_noise = S4Noise.load(saved_model)

Creating right reason mask ... [DONE]


In [12]:
s4_model.noise_model = s4_noise.float()

In [8]:
s4_model.fine_tune_model(
    num_epochs=500,
    learning_rate_planet=1e-3,
    learning_rate_noise=1e-6,
    fine_tune_noise_model=True,
    lean_planet_model=False,
    rotation_grid_down_sample=10,
    upload_rotation_grid=True,
    batch_size=-1)

# Create a residual
tmp_residual_file = s4_model.residuals_dir / \
    Path("01_Residual_Raw_mean_500_BS-1.fits")

tmp_residual = s4_model.compute_residual(
    account_for_planet=False,
    combine="mean")

save_as_fits(
    tmp_residual,
    tmp_residual_file,
    overwrite=True)

  0%|          | 0/500 [00:00<?, ?it/s]